In [1]:
import numpy as np
from CoolProp.CoolProp import PropsSI
import handcalcs.render
from handcalcs.decorator import handcalc
import pint
import endf
from numpy import pi
from numpy import sqrt, log, exp, log10
import os
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
import fluids

ureg = pint.UnitRegistry(auto_reduce_dimensions=True)
ureg.default_format = "~P"
Q_ = ureg.Quantity
ureg.define('USD = [currency] = $')

# Calculating the Heat Flux
The heat flux is used as a boundary condition in OpenFOAM, and can be calculated from the given pin geometry and the core power density

In [20]:
%%render
# Parameters
d_pin = 11.0*ureg('mm') # Pincell diameter
h = 3.0*ureg('m') # Active core height
R_pincell = (17.47/2)*ureg('mm') # Circumradius of hexagonal pincell
q_prime_prime_prime = 100.0*ureg('kW/L') # Voulmetric power density
t_clad = 0.6*ureg('mm') # Clad thickness

<IPython.core.display.Latex object>

In [21]:
%%render
r_pincell = sqrt(3)/2*R_pincell # Pincell inradius - the lattice pitch
A_fuel = pi*(d_pin/2 - t_clad)**2 # Fuel cross sectional area
A_subchannel = 2*sqrt(3)*r_pincell**2 -  pi*(d_pin/2)**2 # Coolant cross sectional area
q = A_fuel*h*q_prime_prime_prime # Power per rod
SA_pin = 2*pi*(d_pin/2)*h # Heated surface area of fuel pin
q_prime_prime = (q/SA_pin).to('W/m^2')

<IPython.core.display.Latex object>

Now, since we're not using chtMultiRegion, OpenFOAM doesn't have access to the thermal conductivity of the clad, which is necessary for determining the temperature gradient from the heat flux, so we must manually divide by $k$ to get the temperature gradient boundary at the clad surface

In [22]:
%%render
# Parameters
k_clad = 15.0*ureg('W/(m*K)')

<IPython.core.display.Latex object>

In [23]:
%%render
TempGradient = q_prime_prime/k_clad

<IPython.core.display.Latex object>

# Postprocessing
Paraview can calculate integrals over axial cross sections, but to calculate surface averaged quantities, we must manually divide by the area.